# KDD Cup 1999 Data

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [93]:
import sklearn
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier 
from sklearn.pipeline import Pipeline
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
from sklearn.utils import resample

In [94]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.


In [95]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [96]:
data = pd.read_csv("data/kddcup.data_10_percent", header=None, names = col_names)

In [97]:
data.shape

(494021, 42)

# 前処理
## カテゴリ化

In [98]:
data.label.value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [99]:
data['label2'] = data.label.where(data.label.str.contains('normal'),'atack')

In [100]:
data.label2.value_counts()

atack      396743
normal.     97278
Name: label2, dtype: int64

In [101]:
data['label3'] = data.label.copy()

In [102]:
data.loc[data.label.str.contains('back|land|neptune|pod|smurf|teardrop|mailbomb|apache2|processtable|udpstorm'),'label3'] = 'DoS'

In [103]:
data.loc[data.label.str.contains('buffer_overflow|loadmodule|perl|rootkit|ps|xterm|sqlattack'),'label3'] = 'U2R'

In [104]:
data.loc[data.label.str.contains('ftp_write|guess_passwd|imap|multihop|phf|spy|warezclient|warezmaster|snmpgetattack|snmpguess|httptunnel|sendmail|named|xlock|xsnoop|worm'),'label3'] = 'R2L'

In [105]:
data.loc[data.label.str.contains('ipsweep|nmap|portsweep|satan|mscan|saint'),'label3'] = 'Probe'

In [106]:
data.label3.value_counts()

DoS        391458
normal.     97278
Probe        4107
R2L          1126
U2R            52
Name: label3, dtype: int64

In [107]:
#joblib.dump(data,'dump/20171118/corrected.pkl')

## サンプリング

In [108]:
#data = resample(data,n_samples=10000,random_state=0)

In [109]:
#data.shape

## 数値化

In [110]:
le_protocol_type = preprocessing.LabelEncoder()

In [111]:
le_protocol_type.fit(data.protocol_type)

LabelEncoder()

In [112]:
data.protocol_type=le_protocol_type.transform(data.protocol_type)

In [113]:
le_service = preprocessing.LabelEncoder()

In [114]:
le_service.fit(data.service)

LabelEncoder()

In [115]:
data.service = le_service.transform(data.service)

In [116]:
le_flag = preprocessing.LabelEncoder()

In [117]:
le_flag.fit(data.flag)

LabelEncoder()

In [118]:
data.flag = le_flag.transform(data.flag)

In [119]:
data.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494021.000000,494021.000000,494021.000000,494021.000000,4.940210e+05,4.940210e+05,494021.000000,494021.000000,494021.000000,494021.000000,...,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000
mean,47.979302,0.467132,23.408894,7.842446,3.025610e+03,8.685324e+02,0.000045,0.006433,0.000014,0.034519,...,232.470778,188.665670,0.753780,0.030906,0.601935,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.746472,0.575606,13.538332,2.250853,9.882181e+05,3.304000e+04,0.006673,0.134805,0.005510,0.782103,...,64.745380,106.040437,0.410781,0.109259,0.481309,0.042133,0.380593,0.380919,0.230590,0.230140
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,14.000000,9.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,14.000000,9.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,42.000000,9.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,2.000000,65.000000,10.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [120]:
data.shape

(494021, 44)

## ラベルの分離

In [121]:
y_test_1 = data.label.copy() 

In [122]:
y_test_2 = data.label2.copy()

In [123]:
y_test_3 = data.label3.copy()

In [124]:
x_test= data.drop(['label','label2','label3'],axis=1)

In [125]:
x_test.shape

(494021, 41)

In [126]:
y_test_1.shape

(494021,)

In [127]:
y_test_2.shape

(494021,)

In [128]:
y_test_3.shape

(494021,)

## 標準化

In [129]:
ss = preprocessing.StandardScaler()

In [130]:
ss.fit(x_test)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [131]:
x_test = ss.transform(x_test)

In [132]:
col_names2 = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

In [133]:
pd.DataFrame(x_test,columns=col_names2).describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,...,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05,4.940210e+05
mean,-3.625574e-14,-5.905329e-13,-7.556595e-14,-1.583611e-13,-1.320064e-14,-5.498890e-14,2.919715e-14,-6.073146e-15,-1.529804e-14,-6.957988e-14,...,-7.353653e-13,1.953080e-13,3.799918e-13,-1.392772e-13,7.581150e-13,2.700207e-13,2.263813e-13,3.603748e-13,6.990594e-13,-9.654619e-14
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-6.779172e-02,-8.115496e-01,-1.729084e+00,-3.484214e+00,-3.061686e-03,-2.628733e-02,-6.673418e-03,-4.772019e-02,-2.571468e-03,-4.413591e-02,...,-3.590542e+00,-1.779188e+00,-1.834994e+00,-2.828667e-01,-1.250621e+00,-1.586293e-01,-4.644176e-01,-4.632024e-01,-2.520395e-01,-2.494640e-01
25%,-6.779172e-02,-8.115496e-01,-6.949824e-01,5.142739e-01,-3.016149e-03,-2.628733e-02,-6.673418e-03,-4.772019e-02,-2.571468e-03,-4.413591e-02,...,3.479668e-01,-1.345391e+00,-8.368938e-01,-2.828667e-01,-1.250621e+00,-1.586293e-01,-4.644176e-01,-4.632024e-01,-2.520395e-01,-2.494640e-01
50%,-6.779172e-02,-8.115496e-01,-6.949824e-01,5.142739e-01,-2.535486e-03,-2.628733e-02,-6.673418e-03,-4.772019e-02,-2.571468e-03,-4.413591e-02,...,3.479668e-01,6.255576e-01,5.993962e-01,-2.828667e-01,8.270476e-01,-1.586293e-01,-4.644176e-01,-4.632024e-01,-2.520395e-01,-2.494640e-01
75%,-6.779172e-02,9.257531e-01,1.373221e+00,5.142739e-01,-2.017381e-03,-2.628733e-02,-6.673418e-03,-4.772019e-02,-2.571468e-03,-4.413591e-02,...,3.479668e-01,6.255576e-01,5.993962e-01,8.323588e-02,8.270476e-01,-1.586293e-01,-4.644176e-01,-4.632024e-01,-2.520395e-01,-2.494640e-01
max,8.234740e+01,2.663056e+00,3.072103e+00,9.585503e-01,7.016400e+02,1.560110e+02,1.498483e+02,2.220663e+01,5.444371e+02,3.831404e+01,...,3.479668e-01,6.255576e-01,5.993962e-01,8.869697e+00,8.270476e-01,2.357583e+01,2.163063e+00,2.162027e+00,4.084676e+00,4.095715e+00


## 学習

In [134]:
clf = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [135]:
t1=time.perf_counter()
clf.fit(x_test, y_test_3)
t2=time.perf_counter()

In [136]:
print(t2-t1,"秒")

67.5275362129978 秒


In [137]:
t1=time.perf_counter()
pred = clf.predict(x_test)
t2=time.perf_counter()

In [138]:
print(t2-t1,"秒")

2.079974817999755 秒


In [139]:
print(classification_report(y_test_3, pred))
print(confusion_matrix(y_test_3, pred))

             precision    recall  f1-score   support

        DoS       1.00      1.00      1.00    391458
      Probe       0.99      0.99      0.99      4107
        R2L       0.97      0.88      0.92      1126
        U2R       0.60      0.50      0.55        52
    normal.       1.00      1.00      1.00     97278

avg / total       1.00      1.00      1.00    494021

[[391413      2      0      0     43]
 [     1   4086      0      0     20]
 [     0      0    995      1    130]
 [     0      0      3     26     23]
 [     0     22     31     16  97209]]


In [141]:
#joblib.dump(clf,'dump/20171118/MLPClassifier10per.pkl')

['dump/20171118/MLPClassifier10per.pkl']